# LLM as a judge: 

This notebook demonstrates an evaluation pipeline where an LLM judges how well AI responses align with paper hypotheses.

In [1]:
%reload_ext autoreload
%autoreload 2

import configparser

from IPython.display import Markdown
from openai import OpenAI

config = configparser.ConfigParser()
config.read("../secrets.ini")

['../secrets.ini']

In [2]:
API_KEY = config["DEFAULT"]["API_KEY"]
client = OpenAI(api_key=API_KEY)

In [3]:
def generate_response(
    ai_response: str,
    paper_key_findings: dict,
    client: OpenAI,
    model: str = "gpt-4o",
    temperature: float = 0.9,
) -> str:
    """Generate a response using the OpenAI API with formatted prompt.

    Parameters
    ----------
    ai_response : str
        The AI response to include in the prompt
    paper_key_findings : dict
        The key findings from the paper to include in the prompt
    client : OpenAI client
        The initialized OpenAI client
    model : str, optional
        The model to use for the API call, by default "gpt-4o"
    temperature : float, optional
        Temperature parameter for response generation, by default 0.9

    Returns
    -------
    str
        The content of the model's response

    """
    prompt = """\
    # Instruction

    Your task is to compare the analysis of proteomics data from an original research paper with an AI-generated response. You will need to match corresponding positions and findings between the two analyses, identifying which parts of the AI analysis are present in the original research paper.

    # Evaluation Process

    ## Matching Procedure

    We will provide you with a dictionary that contains the key findings of an original research paper, which serve you as a ground truth.

    STEP 1: Extract key claims and identified protein patterns from the AI-generated response and write them down.

    STEP 2: Compare the key findings of the original research paper with the key claims extracted in step 1 and the AI-generated response itself. Categorise the matches as: Aligned, Partially Aligned, Misaligned, Not Addressed.

    STEP 3: Create a matching table with the following columns:
    * Original Paper Finding
    * Citation from Corresponding AI Analysis
    * Alignment Assessment (Aligned, Partially Aligned, Misaligned, Not Addressed)
    * Notes on Differences or Similarities

    STEP 4: Provide an overall assessment of how well the AI analysis captured the original paper's findings.

    ## Input Materials

    ### Dictionary with key findings from original research paper

    {paper_key_findings}

    ### AI-Generated Analysis

    {ai_response}

    ## Output Format

    ### Matching Table

    | Original Paper Finding | Citation from Corresponding AI Analysis | Alignment Assessment | Notes |
    |------------------------|---------------------------|----------------------|-------|
    | Hypothesis 1: [Description] | [AI phrase that matches] | [Aligned/Partially/Misaligned] | [Explanation] |
    | Hypothesis 2: [Description] | [AI phrase that matches] | [Aligned/Partially/Misaligned] | [Explanation] |
    | [etc...] | | | |

    ### Findings Not Addressed

    List any significant findings from the original paper that were not addressed in the AI analysis.

    ### AI Claims Without Paper Support**

    List any claims made in the AI analysis that are not supported by the original paper excerpt.

    ### Overall Assessment

    Provide a summary assessment of how well the AI analysis matched and interpreted the original paper findings, noting strengths and weaknesses in the correspondence.

    ### Task Overview

    Your task is to compare the original analysis from the research paper with the AI-generated analysis of the same proteomics data. You need to identify corresponding points between the two analyses and evaluate how well the AI interpretation aligns with the scientific findings from the paper."""

    formatted_prompt = prompt.format(
        ai_response=ai_response, paper_key_findings=paper_key_findings
    )

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are an expert evaluator specializing in proteomics and molecular biology analysis.",
            },
            {"role": "user", "content": formatted_prompt},
        ],
        temperature=temperature,
    )

    return response.choices[0].message.content

In [4]:
paper_key_findings = {
    "Hypothesis1": "The proteasome complex is downregulated.",
    "Hypothesis2": "Heat shock proteins are upregulated.",
    "Hypothesis3": "The (pre-)ribosome is downregulated.",
    "Hypothesis4": "Cytosolic protein folding machinery is upregulated.",
    "Hypothesis5": "Regulators of proteasome assembly are upregulated.",
    "Hypothesis6": "Proteins involved in sterol synthesis are upregulated.",
    "Hypothesis7": "Multiple targets of the unfolded protein response are upregulated.",
    "Hypothesis10": "Protein homeostatis is disrupted, as evident from increased protein folding efforts and decreased protein synthesis.",
    "Hypothesis11": "Reduced proteasome capacity overly affects ERAD substrates.",
}

ai_response = """
# Analysis of RPN4 Knockout Strain

## 1. Functional Analysis

* **Protein Synthesis & Ribosome Biogenesis (Downregulated):** Many ribosomal proteins (RPLs, RPSs, etc.) and factors involved in ribosome assembly and function, such as NSA1 and DED1, are downregulated. This suggests a reduction in protein synthesis capacity and altered ribosome biogenesis in the RPN4 knockout strain. This may affect the cell's ability to translate proteins efficiently.

* **Proteasome and Proteasome Regulatory Proteins (Downregulated):** Several proteins associated with the proteasome complex, like PRE1, RPN1, and RPN6, are downregulated. This could indicate altered protein degradation mechanisms, which might affect protein turnover and stress response.

* **Heat Shock Proteins (Upregulated):** Proteins like HSP82, SSA1, and HSP104 are upregulated, suggesting an activated stress response, likely compensating for proteostasis imbalance.

* **Metabolic Enzymes (Upregulated):** Enzymes like TDH1 (glyceraldehyde-3-phosphate dehydrogenase) and CIT1 (citrate synthase) are upregulated, hinting at alterations in glycolysis and the TCA cycle, potentially adapting to metabolic needs in the altered cellular state due to RPN4 knockout.

## 2. Ontology Analysis

* **Stress Response and Heat Shock Response (Upregulated):** The upregulation of heat shock proteins implies activation of cellular mechanisms to maintain protein stability under stress conditions.

* **Ribosome Biogenesis and Assembly (Downregulated):** Enrichment in downregulated ribosomal proteins reveals diminished synthesis capacity impacting overall cellular growth and proliferation processes.

* **Metabolic Pathways (Upregulated):** Increased expression of metabolic enzymes suggests a shift in cellular metabolism, possibly enhancing pathways to meet energetic or biosynthetic needs under the RPN4-deleted condition.

## 3. Critical Review

* There is consistency across the analyses indicating a downregulation in protein synthesis capability accompanied by compensatory mechanisms (upregulated stress proteins).

* The simultaneous downregulation of proteasome components alongside ribosomal subunits suggests a strategic slowdown in overall protein turnover and synthesis, enabling the cell to handle stress effectively.

## 4. Biological Context

* **Experimental Design:** The comparison examines the RPN4 knockout against wildtype yeast, focusing on initial cellular responses at zero hours post-variable introduction.

* **Biological Implications:** The increased stress response, downregulated protein synthesis, and highlighted metabolic adjustments indicate cellular adaptation strategies to maintain homeostasis in response to genetic deletion of RPN4.

## 5. Focused Analysis

Key aspects worth investigating further:

1. **Heat Shock Response Pathway:** Given its role in managing protein unfolding and aggregation under stress, further exploring this could uncover strategies to mitigate proteotoxic stress.

2. **Metabolic Adaptation in Central Metabolism:** Understanding how glycolysis and TCA cycle enzymes adapt provides insights into metabolic flexibility and energy homeostasis.

3. **Ribosome and Proteasome Interplay:** Analyzing how these complexes are coregulated under stress or genetic modifications offers insight into protein homeostasis control mechanisms.

4. **RPN4's Role in Regulation:** Understanding RPN4's network regulations could clarify its overarching influence on cellular stress mechanisms and adaptation pathways.

Given these aspects, please specify which area you would like to delve further into, and I can perform a deeper, focused analysis in that area.
"""

evaluation = generate_response(
    ai_response,
    paper_key_findings,
    client,
)

Markdown(evaluation)

### Matching Table

| Original Paper Finding | Citation from Corresponding AI Analysis | Alignment Assessment | Notes |
|------------------------|----------------------------------------|----------------------|-------|
| Hypothesis 1: The proteasome complex is downregulated. | "Proteasome and Proteasome Regulatory Proteins (Downregulated): Several proteins associated with the proteasome complex, like PRE1, RPN1, and RPN6, are downregulated." | Aligned | Direct match indicating a decrease in proteasome complex proteins. |
| Hypothesis 2: Heat shock proteins are upregulated. | "Heat Shock Proteins (Upregulated): Proteins like HSP82, SSA1, and HSP104 are upregulated." | Aligned | Direct match showing increased expression of heat shock proteins. |
| Hypothesis 3: The (pre-)ribosome is downregulated. | "Protein Synthesis & Ribosome Biogenesis (Downregulated): Many ribosomal proteins (RPLs, RPSs, etc.) and factors involved in ribosome assembly and function are downregulated." | Aligned | Clear reference to downregulation of ribosomal proteins, matching the hypothesis. |
| Hypothesis 4: Cytosolic protein folding machinery is upregulated. | "Stress Response and Heat Shock Response (Upregulated): The upregulation of heat shock proteins implies activation of cellular mechanisms to maintain protein stability under stress conditions." | Partially Aligned | While heat shock proteins are mentioned, direct mention of cytosolic folding machinery is less clear.|
| Hypothesis 5: Regulators of proteasome assembly are upregulated. | Not explicitly mentioned. | Not Addressed | The AI analysis doesn't mention upregulation of proteasome assembly regulators. |
| Hypothesis 6: Proteins involved in sterol synthesis are upregulated. | Not explicitly mentioned. | Not Addressed | The AI analysis does not cover sterol synthesis pathways. |
| Hypothesis 7: Multiple targets of the unfolded protein response are upregulated. | "Heat Shock Response Pathway: Given its role in managing protein unfolding and aggregation under stress..." | Partially Aligned | Only mentions management of protein unfolding, not specific targets of the UPR. |
| Hypothesis 10: Protein homeostasis is disrupted, as evident from increased protein folding efforts and decreased protein synthesis. | "The simultaneous downregulation of proteasome components alongside ribosomal subunits suggests a strategic slowdown in overall protein turnover and synthesis..." | Partially Aligned | General notion of disrupted homeostasis is captured, specific mentions of increased protein folding efforts are indirect. |
| Hypothesis 11: Reduced proteasome capacity overly affects ERAD substrates. | Not explicitly mentioned. | Not Addressed | AI analysis lacks discussion on how ERAD substrates are impacted. |

### Findings Not Addressed

- Hypothesis 5: Regulators of proteasome assembly are not mentioned.
- Hypothesis 6: Proteins involved in sterol synthesis are not covered.
- Hypothesis 11: No mention of how reduced proteasome capacity affects ERAD substrates.

### AI Claims Without Paper Support

- "Metabolic Enzymes (Upregulated):" The AI analysis discusses metabolic enzyme upregulation, which is not mentioned in the original findings.

### Overall Assessment

The AI analysis generally aligns well with the original paper's findings, accurately capturing the downregulation of the proteasome complex and ribosome biogenesis, as well as the upregulation of heat shock proteins. However, it does not address several specific findings related to proteasome assembly regulators, sterol synthesis, and ERAD substrate interactions. Additionally, the AI introduces unsupported claims about metabolic enzyme changes. This suggests the AI can effectively generalize major themes but may miss specific findings or introduce unrelated analysis aspects. Strengths include capturing core findings; weaknesses involve missing particular insights and introducing unsupported claims.

In [5]:
ai_response = """
To provide a thorough analysis of the proteomics data received from your proteomics experiment comparing the RPN4 knockout strain to wild type yeast, we will go through each step of analysis outlined in the provided structure.

## 1. Functional Analysis:

### Upregulated Proteins:

- **Chaperones and Heat Shock Proteins** (HSP82, HSC82, STI1, HSP26, KAR2): Involvement in protein folding and stress response. These proteins are often involved in cellular response to stress conditions and may facilitate protein refolding during stress or damage [PMID:27365453].
- **Metabolic Enzymes** (TDH1, CIT1, ENO1, HXK1, GLK1, SDH2): Enzymes involved in glycolytic pathways, metabolic processes, and the tricarboxylic acid (TCA) cycle. Their upregulation suggests increased cellular metabolism or compensatory mechanisms due to perturbations in energy production.
- **Proteases and Protein Modifying Enzymes** (PEP4, PRB1, PBI2, APE1): Involve in protein turnover and degradation, reflecting possible stress adaptation or altered protein quality control requirements in the RPN4 knockout.
- **Antioxidant and Stress Response Proteins** (PRX1, TSA1): Part of the defense system against oxidative stress, their upregulation can be indicative of increased reactive oxygen species (ROS) management.
- **Transposon and Mobile Elements** (TY1B-PR1, TY2B-LR1): Often stress-responsive, their upregulation points to genomic instability or retrotransposon activity under stress conditions.

### Downregulated Proteins:

- **Ribosomal Proteins and Translation Machinery** (RPL25, RPS9B, RPS13): Downregulation suggests a general reduction in protein synthesis, possibly as a cellular tradeoff in response to stress conditions or as part of a wider adaptive response possibly yielding energy for other processes.
- **Proteasome Subunits** (PRE1, PRE9, PRE8, RPN3): This implies a possible decrease in proteasomal protein degradation pathways, which could result in protein accumulation or functional shifts in protein catabolism.
- **Transport and Trafficking Proteins** (RPT5, RPN2): Suggestive of impaired protein and organelle transport, potentially affecting cellular protein recycling or activation pathways.
- **Energy Metabolism Components** (HXT2, URA1): Loss of competence in glucose transport and related metabolic adaptations could imply an energetic shift or a metabolic bottleneck.

## 2. Ontology Analysis:

- **Proteasome Complex Pathways** (GO:CC, KEGG): Strongly represented, indicating the proteasome's role critically underlies differential response, possibly compensating for increased misfolded protein presence or oxidative damage.
- **Chaperone and Folding Pathways** (GO:MF): Highlighting increased protein refolding and stress adaptation processes, possibly as a compensatory reaction to proteostasis imbalance.
- **Ribosomal and Translation Inhibition** (WP): Correlates with downregulated ribosomal proteins and implies possible overall reduction of protein synthesis effectiveness under RPN4 knockout conditions.

## 3. Critical Review:

The analyses of upregulated and downregulated proteins display congruent patterns with enrichment results, primarily reflecting adaptations in protein folding, degradation, and possible biosynthetic shifts toward survival under stress. There are no notable contradictions evident in the results.

## 4. Biological Context:

The observed changes, particularly in proteostasis regulators such as proteasomes and chaperones, suggest compensatory stress responses in protein damage control. The decrease in translational machinery components suggests a shift in cellular resource allocation priorities, potentially lowering energy expenditure or reallocating resources to manage oxidative stress more effectively. This observation is consistent with a cellular response prioritization strategy, which could influence cell growth, division, and survival under altered environmental or genetic conditions.

## 5. Focused Analysis Recommendations:

- **Proteasome-Related Pathways**: Given their crucial roles in proteostasis and stress response, further exploration into proteasome dynamics might reveal deeper insights into cellular priority shifts in RPN4 knockout.
- **Heat Shock Response**: Understanding the specifics behind chaperone and heat shock protein upregulation may lead to novel regulatory mechanisms underpinning stress responses.
- **Energy Metabolism Dynamics**: Investigation into changes in energy-producing pathways such as glycolysis and TCA cycle, given the altered expression of corresponding enzymes.
- **Translational Machinery**: Reduced ribosomal proteins implicate translational control which can impact cell growth, necessitating in-depth study of ribosome biosynthesis and function.
- **Retrotransposition Activities**: Implications of mobile element activation as part of stress-induced genomic stability responses provide a novel angle on cellular adaptability under perturbation conditions.

I recommend beginning the deeper analysis focusing on the Proteasome-Related Pathways due to their central role in mediating proteostasis and potential broader implications on cellular homeostasis. Would you like to start with this aspect, or is there another focus area you prefer to investigate further?
"""

evaluation = generate_response(
    ai_response,
    paper_key_findings,
    client,
)

Markdown(evaluation)

### Matching Table

| Original Paper Finding                                          | Citation from Corresponding AI Analysis                                                                                       | Alignment Assessment | Notes                                                                                                                                                                                                 |
|-----------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------|----------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Hypothesis 1: The proteasome complex is downregulated.          | **Proteasome Subunits** (PRE1, PRE9, PRE8, RPN3): This implies a possible decrease in proteasomal protein degradation pathways | Aligned              | The AI correctly identifies the downregulation of proteasome subunits, reflecting a decrease in proteasomal protein degradation.                                                                                          |
| Hypothesis 2: Heat shock proteins are upregulated.              | **Chaperones and Heat Shock Proteins** (HSP82, HSC82, STI1, HSP26, KAR2): Involvement in protein folding and stress response  | Aligned              | The AI highlights the upregulation of heat shock proteins, consistent with the original paper's finding.                                                                                                         |
| Hypothesis 3: The (pre-)ribosome is downregulated.              | **Ribosomal Proteins and Translation Machinery** (RPL25, RPS9B, RPS13): Downregulation suggests a general reduction in protein synthesis | Aligned              | The downregulation of ribosomal proteins in the AI analysis matches the original paper's observation of downregulated (pre-)ribosomes.                                                                 |
| Hypothesis 4: Cytosolic protein folding machinery is upregulated. | **Chaperones and Heat Shock Proteins**: Involvement in protein folding and stress response                                    | Aligned              | The AI's mention of chaperones and heat shock proteins suggests an upregulation in protein folding machinery, in line with the original paper's findings.                                                                                   |
| Hypothesis 5: Regulators of proteasome assembly are upregulated. | Not specifically addressed                                                                                                   | Not Addressed        | The AI analysis does not mention the upregulation of regulators specifically related to proteasome assembly.                                                                                      |
| Hypothesis 6: Proteins involved in sterol synthesis are upregulated. | Not mentioned                                                                                                                 | Not Addressed        | The AI does not address the upregulation of proteins involved in sterol synthesis.                                                                                                                                                                              |
| Hypothesis 7: Multiple targets of the unfolded protein response are upregulated. | Mention of increased reactive oxygen species (ROS) management through **Antioxidant and Stress Response Proteins** (PRX1, TSA1) | Partially Aligned    | While it touches on stress response, it doesn't specifically mention the unfolded protein response targets being upregulated.                                                                                     |
| Hypothesis 10: Protein homeostasis is disrupted, as evident from increased protein folding efforts and decreased protein synthesis. | The mention of **Chaperones and Heat Shock Proteins** and **Ribosomal Proteins and Translation Machinery** suggests this.     | Aligned              | The AI captures the disruption of protein homeostasis through increased folding efforts and reduced synthesis, aligning with the original finding.                                                                                           |
| Hypothesis 11: Reduced proteasome capacity overly affects ERAD substrates. | Not specifically mentioned                                                                                                    | Not Addressed        | The AI analysis did not address the effect of reduced proteasome capacity on ERAD substrates.                                                                                                                                                 |

### Findings Not Addressed

1. Regulators of proteasome assembly are upregulated.
2. Proteins involved in sterol synthesis are upregulated.
3. Reduced proteasome capacity overly affects ERAD substrates.

### AI Claims Without Paper Support

1. **Metabolic Enzymes**: Upregulation of TDH1, CIT1, ENO1, HXK1, GLK1, SDH2, suggesting increased cellular metabolism. This specific claim was not mentioned in the original paper findings.
2. **Transposon and Mobile Elements**: TY1B-PR1, TY2B-LR1 upregulation related to stress response was not supported by the original paper.

### Overall Assessment

The AI-generated analysis aligns well with several major findings from the original research paper, particularly regarding the downregulation of the proteasome complex and ribosomal proteins, and the upregulation of chaperones and heat shock proteins involved in stress response and protein folding. However, it misses key points regarding the upregulation of regulators of proteasome assembly, proteins involved in sterol synthesis, and the effect on ERAD substrates. Additionally, the AI introduces some claims not covered by the original paper, such as the upregulation of specific metabolic enzymes and transposons, which may indicate overinterpretation or inclusion of unrelated data. Overall, the AI analysis captures the general theme of proteostasis disruption but lacks precision in aligning with all specific findings of the original research paper.